# ARKitScenes dataset processing

Please run this notebook to download, pre-process, and also create necessary files and information for training on ZoeDepth model on ARkitScenes dataset.

In [ ]:
import os
import shutil
import subprocess
import collections
import cv2
import numpy as np

subset = ['raw', 'upsampling', 'intrinstics']
ARKIT_PATH = 'path/data/ARKitScenes'
split = ['Training', 'Validation']

In [4]:
training_vid_ids = os.listdir(os.path.join(ARKIT_PATH, subset[1], 'Training'))
validation_vid_ids = os.listdir(
    os.path.join(ARKIT_PATH, subset[1], 'Validation'))

print(len(training_vid_ids) + len(validation_vid_ids))
all_vid_ids = training_vid_ids + validation_vid_ids

2257


In [5]:
count = 0
for sample in training_vid_ids:
        count += len(os.listdir(os.path.join(ARKIT_PATH, subset[1], 'Training', sample, 'highres_depth')))
        
print(count)
# 5599 + 39000 = 44599

39000


# Download dataset
First Please clone the ARKitScenes dataset from the following link: <a href="https://github.com/cake-lab/ARKitScenes">ARkitScenes</a>

And then change the path to the download_data.py file.

In [ ]:
for id in training_vid_ids:
    args = ["python", "path/download_data.py", "raw", "--split","Training",
            "--video_id", id, "--download_dir", "dataset_download", "--raw_dataset_assets","lowres_wide"]

    subprocess.run(args)

# Check if dataset to sample ids downloaded completely

In [4]:
sum_upsampling = os.listdir(os.path.join(ARKIT_PATH, subset[1], 'Training')) + os.listdir(os.path.join(ARKIT_PATH, subset[1], 'Validation'))
sum_raw = os.listdir(os.path.join(ARKIT_PATH, subset[0], subset[0], 'Training')) + os.listdir(os.path.join(ARKIT_PATH, subset[0], subset[0], 'Validation'))

compare = lambda x, y: collections.Counter(x) == collections.Counter(y)

if compare(sum_upsampling, sum_raw):
    print('Data downloaded successfully!')
else:
    print('Sample IDs do not match!')

Sample IDs do not match!


## Copying corresponding lowres frames to highres folder

In [62]:
# samples_dir_dest = os.path.join(dist, split[0])
upsamples_dir_ARKit = os.path.join(ARKIT_PATH, subset[1])
raw_dir_ARKit = os.path.join(ARKIT_PATH, subset[0], subset[0])

In [66]:
def create_dataset(split, upsamples_dir_ARKit, raw_dir_ARKit):
    upsamples_dir_ARKit = os.path.join(upsamples_dir_ARKit, split)
    raw_dir_ARKit = os.path.join(raw_dir_ARKit, split)

    for sample in os.listdir(upsamples_dir_ARKit):
        count = 0
        total_frames = len(os.listdir(os.path.join(upsamples_dir_ARKit, sample,'highres_depth')))
        # os.mkdir(os.path.join(upsamples_dir_ARKit, sample,'lowres_wide'))
        for frame in os.listdir(os.path.join(upsamples_dir_ARKit, sample,'highres_depth')):
            frame_path = os.path.join(raw_dir_ARKit, sample, 'lowres_wide', frame)
            if os.path.exists(frame_path):
                dest = os.path.join(upsamples_dir_ARKit, sample,'lowres_wide', frame)
                shutil.copy(frame_path, dest) # uncomment to copy files
                count +=1

        print('Found {}/{} frames for sample {}'.format(count,total_frames, sample))
        print('-------------------------------------------------')

In [ ]:
create_dataset(split[1], upsamples_dir_ARKit, raw_dir_ARKit)
create_dataset(split[0], upsamples_dir_ARKit, raw_dir_ARKit)

# Adding camera parameters to the dataset

In [34]:
cam_parameters = os.path.join(ARKIT_PATH , subset[2], 'raw')
upsampling_path = os.path.join(ARKIT_PATH , subset[1])

for sample in os.listdir(os.path.join(cam_parameters,split[0])):
    # check if the sample exist in upsampling
    if sample in os.listdir(os.path.join(upsampling_path,split[0])):
        # copy every thing in os.path.join(cam_parameters,split[1],sample) to os.path.join(upsampling_path,split[1],sample)
        
        if not os.path.exists(os.path.join(upsampling_path,split[0],sample, 'lowres_wide_intrinsics')):
            os.makedirs(os.path.join(upsampling_path,split[0],sample, 'lowres_wide_intrinsics'))
            
        for file in os.listdir(os.path.join(cam_parameters,split[0],sample, 'lowres_wide_intrinsics')):
            # make new directory
            shutil.copy(os.path.join(cam_parameters,split[0],sample,'lowres_wide_intrinsics',file), os.path.join(upsampling_path,split[0],sample, 'lowres_wide_intrinsics'))
            
            

# Calculating the information for adding to config file
For training the zoedepth model.

## File names txt

In [1]:
def create_files_input(split, file_name,upsamples_dir_ARKit):
    upsamples_dir_ARKit = os.path.join(upsamples_dir_ARKit, split)
    print(upsamples_dir_ARKit)
    for sample in os.listdir(upsamples_dir_ARKit):
        training_frame_path = os.path.join(upsamples_dir_ARKit, sample,'lowres_wide')
        gt_frame_path =  os.path.join(upsamples_dir_ARKit, sample,'downsampled_gt')

        training_frames = os.listdir(training_frame_path)
        gt_frames= os.listdir(gt_frame_path)

        for i, frame_train in enumerate(training_frames):
            print(os.path.join(sample,'lowres_depth',frame_train))
            print(os.path.join(sample,'downsampled_gt',gt_frames[i]))

            with open(file_name, 'a') as f:
                f.write(os.path.join(sample,'lowres_wide', frame_train) + ' ' + os.path.join(sample,'downsampled_gt',gt_frames[i]) + '\n')

    print('Done!')

# Train/Val splits
run this to generate the train/val splits for training and evaluation on ARkitScenes

In [5]:
upsamples_dir_ARKit = os.path.join(ARKIT_PATH, subset[1])
create_files_input(split[0], './train_test_inputs/arkit_scenes_train_files_with_gt.txt', upsamples_dir_ARKit)

upsamples_dir_ARKit = os.path.join(ARKIT_PATH, subset[1])
create_files_input(split[1], './train_test_inputs/arkit_scenes_test_files_with_gt.txt' ,upsamples_dir_ARKit)

/mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation
47334105/lowres_depth/47334105_93033.275.png
47334105/downsampled_gt/47334105_93033.275.png
47334105/lowres_depth/47334105_92946.510.png
47334105/downsampled_gt/47334105_92946.510.png
47334105/lowres_depth/47334105_93019.381.png
47334105/downsampled_gt/47334105_93019.381.png
47334105/lowres_depth/47334105_92896.431.png
47334105/downsampled_gt/47334105_92896.431.png
47334105/lowres_depth/47334105_92972.800.png
47334105/downsampled_gt/47334105_92972.800.png
47334105/lowres_depth/47334105_92990.693.png
47334105/downsampled_gt/47334105_92990.693.png
47334105/lowres_depth/47334105_92888.934.png
47334105/downsampled_gt/47334105_92888.934.png
47334105/lowres_depth/47334105_93017.282.png
47334105/downsampled_gt/47334105_93017.282.png
47334105/lowres_depth/47334105_93005.487.png
47334105/downsampled_gt/47334105_93005.487.png
47334105/lowres_depth/47334105_93046.070.png
47334105/downsampled_gt/47334105_93046.070.png
47334105/lowres_dep

## Max and Min depth values

In [8]:
def find_min_max_depth(split):
    min = 100000
    max = 0
    print(os.path.join(ARKIT_PATH, subset[1], split))
    for sample in os.listdir(os.path.join(ARKIT_PATH, subset[1], split)):
        for depth in os.listdir(os.path.join(os.path.join(ARKIT_PATH, subset[1], split, sample, 'downsampled_gt'))):
            # find min and max depth
            depth_path = os.path.join(os.path.join(ARKIT_PATH, subset[1], split, sample, 'downsampled_gt', depth))
            depth_img = cv2.imread(depth_path, cv2.IMREAD_ANYDEPTH)
            if max < np.max(depth_img):
                max = np.max(depth_img)
            if min > np.min(depth_img):
                min = np.min(depth_img)
    return (min, max) # add this values to the config file

In [9]:
minmax1 = find_min_max_depth(split[0])
minmax2 = find_min_max_depth(split[1])

/mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Training
/mnt/IRONWOLF1/ashkan/data/ARKitScenes/upsampling/Validation


In [10]:
print(minmax2)
print(minmax1)

(0, 42263)
(0, 48340)


# Add downsampled subset

In [25]:
upsamples_dir_ARKit = os.path.join(ARKIT_PATH, subset[1], split[0]) # change the subset based on your needs
print(split[0])
for sample in os.listdir(upsamples_dir_ARKit):
    # try:
    #     os.mkdir(os.path.join(upsamples_dir_ARKit, sample, 'downsampled_gt'))
    # except:
    #     continue
    for frame in os.listdir(os.path.join(os.path.join(upsamples_dir_ARKit, sample, 'highres_depth'))):
        highres_depth = cv2.imread(os.path.join(os.path.join(upsamples_dir_ARKit, sample, 'highres_depth', frame)), cv2.IMREAD_ANYDEPTH)
        lowres_depth = cv2.imread(os.path.join(os.path.join(upsamples_dir_ARKit, sample, 'lowres_depth', frame)), cv2.IMREAD_ANYDEPTH)
        downsampled_GT_depth = cv2.resize(
            highres_depth, (lowres_depth.shape[1], lowres_depth.shape[0]), interpolation=cv2.INTER_NEAREST)
        
        cv2.imwrite(os.path.join(os.path.join(upsamples_dir_ARKit, sample, 'downsampled_gt', frame)), downsampled_GT_depth)

Training
